In [1]:
import numpy as np
import matplotlib.pyplot as plt

import parmed
from parmed.charmm import CharmmParameterSet
from parmed.formats.registry import load_file

%matplotlib inline


In [2]:
psf_file = 'inp_files/da.psf'
pdb_file = 'inp_files/smd_ini.pdb'
prm_file = 'inp_files/par_all27_prot_lipid_cmap.prm'

In [3]:
prm = CharmmParameterSet.load_set(pfile = prm_file)
psf = load_file(psf_file)
pdb = load_file(pdb_file)

HERE
READING



/home/shaunak/anaconda3/envs/MD/lib/python3.6/site-packages/parmed/charmm/parameters.py:754: ParameterWarning: Atom type C not present in AtomType list
  warnings.warn('Atom type %s not present in AtomType list' % key, ParameterWarning)


## Energy Calculation

In [4]:
class Energies:
    def __init__(self, psf, pdb, prm):
        self.psf = psf
        self.pdb = pdb
        self.prm = prm
        
        ## initializing bond types and coordinates
        coords = pdb.coordinates
        bonds = psf.bonds
        angles = psf.angles
        dihedrals = psf.dihedrals
        impropers = psf.impropers
        for bond in bonds:
            ## bond types
            tid1 = bond.atom1.type
            tid2 = bond.atom2.type
            
            t = (tid1, tid2)
            bond.type = prm.bond_types[t]
    
            ## coordinates
            b1 = bond.atom1
            b1.xx, b1.xy, b1.xz = coords[b1.idx]
            
            b2 = bond.atom2
            b2.xx, b2.xy, b2.xz = coords[b2.idx]
            
            ## non bonded parameters eps and rmin
            name = b1.type
            eps, rmin, _, _ = prm.nonbonded_types[name]
            b1.rmin = rmin
            b1.epsilon = eps
            
            name = b2.type
            eps, rmin, _, _ = prm.nonbonded_types[name]
            b2.rmin = rmin
            b2.epsilon = eps
        
            
            
        ## initializing angle types
        for angle in angles:
            a1 = angle.atom1.type
            a2 = angle.atom2.type
            a3 = angle.atom3.type
            
            t = (a1, a2, a3)
            
            angle.type = prm.angle_types[t]
            a = angle.measure()
            a0 = angle.type
            
        for i, dihedral in enumerate(dihedrals):
            a1 = dihedral.atom1.type
            a2 = dihedral.atom2.type
            a3 = dihedral.atom3.type
            a4 = dihedral.atom4.type
            
            t = (a1, a2, a3, a4)
            g = self._get_bond_type(t, prm.dihedral_types)
            dihedrals[i].type = prm.dihedral_types[g]
            
        for i, improper in enumerate(impropers):
            a1 = improper.atom1.type
            a2 = improper.atom2.type
            a3 = improper.atom3.type
            a4 = improper.atom4.type
            
            t = (a1, a2, a3, a4)
            g = self._get_bond_type(t, prm.improper_types)
            impropers[i].type = prm.improper_types[g]
        
    def bond(self):
        bonds = psf.bonds
        s = 0
        for bond in bonds:
            r1 = pdb.coordinates[bond.atom1.idx]
            r2 = pdb.coordinates[bond.atom2.idx]
            k = bond.type.k
            dist = np.linalg.norm(r1 - r2)
            
            r0 = bond.type.req
            cust_energy = k * ((dist - r0)**2)
            s += cust_energy
            
        return s    
    
    def angle(self):
        s = 0
        
        for angle in psf.angles:
            theta = angle.measure()
            theta0 = angle.type.theteq
            k = angle.type.k
            theta_diff = (np.pi * (theta - theta0)) / 180
            en = k * (theta_diff ** 2)
#             print(en, angle.energy())
            s += en
        return s
    
    ## DIHEDRAL AND IMPROPER HELPER FUNCTION
    def _get_bond_type(self, t, search_list):
        '''
        Input :
        t : a 4-atom tuple, 
        search_list: usually prm.dihedral_types or prm.improper_types

        Return type: BondType
        '''
        for dtype in search_list:
            val = True
            for i in range(4):
                val = val and (dtype[i] == t[i] or dtype[i] == 'X')

            if val:
                return dtype

            ## Attempt reverse search
            val = True
            for i in range(4):
                val = val and (dtype[i] == t[3 - i] or dtype[i] == 'X')

            if val:
                return dtype
        return None
    
    def dihedral(self):
        dihedrals = psf.dihedrals
        k_sum = 0
        for dihed in dihedrals:
            dtype = dihed.type
            
            total_energy = 0
            chi = dihed.measure() * (np.pi / 180)
            if chi > np.pi or chi < -np.pi:
                print("Houston, we have a problem")
#             print(dihed.measure())
            if isinstance(dtype, parmed.DihedralType):
                n = dtype.per
                kchi = dtype.phi_k
                phase = dtype.phase * (np.pi / 180)
                en = kchi * (1 + np.cos(n * chi + phase))
                total_energy += en
                
            elif isinstance(dtype, parmed.DihedralTypeList):
                e = 0
                for term in dtype:
                    n = term.per
                    kchi = term.phi_k
                    phase = term.phase * (np.pi / 180)
#                     print(n)
#                     print(kchi, n, chi, phase)
                    if n > 0:
                        en = kchi * (1 + np.cos(n * chi - phase))
#                         print(np.cos(n * chi - phase), n * chi - phase)
                    else:
                        en = kchi * (chi - phase)**2
                    k_sum += kchi
                    print(en)
                    e += np.abs(en)
                total_energy += np.abs(e)
                
            else:
                printf("Cant determine parameters for {}".format(t))
                return 0
        print("2k : ", 2 * k_sum)
        return total_energy
    
    def improper(self):
        impropers = psf.impropers
        total_energy = 0
        for imp in impropers:
            psi = imp.measure() * (np.pi / 180)
            dtype = imp.type
            psi_0 = dtype.psi_eq * (np.pi / 180)
            k = dtype.psi_k 
            en = k *(psi - psi_0)**2
            
            total_energy += en
            
        return total_energy
    
    # NON BONDED
    def LJ(self):
        energy = 0
        coords = psf.coordinates
        for i, a in enumerate(psf.atoms):
            for j, b in enumerate(psf.atoms):
                if j <= i: 
                    continue
                eps_ij = np.sqrt(a.epsilon * b.epsilon)
                r_ij = (a.rmin + b.rmin) / 2
    
                r1 = coords[a.idx]
                r2 = coords[b.idx]
                r = np.linalg.norm(r1 - r2)
                ratio = r_ij / r
                en = 4 * eps_ij * (ratio**12 - (ratio**6))
                energy += en
                
        return energy
    
    def Elec(self):
        energy = 0
        coords = psf.coordinates
        for i, a in enumerate(psf.atoms):
            for j, b in enumerate(psf.atoms):
                if j <= i: 
                    continue
                k = 8.9 * 1e9
                r1 = coords[a.idx]
                r2 = coords[b.idx]
                r = np.linalg.norm(r1 - r2)
                q1 = a.charge
                q2 = b.charge
                en = k * q1 * q2 / (r * 1e-9)
                energy += en
        return energy
    


In [5]:
h = Energies(psf, pdb, prm)
# h.dihedral()

## Force calculation : Resources used in `resources` folder

In [6]:
from parmed.geometry import angle

class Forces:
    def __init__(self, psf, pdb, prm):
        self.psf = psf
        self.pdb = pdb
        self.prm = prm
        
        ## initializing bond types and coordinates
        coords = pdb.coordinates
        bonds = psf.bonds
        angles = psf.angles
        dihedrals = psf.dihedrals
        impropers = psf.impropers
        for bond in bonds:
            ## bond types
            tid1 = bond.atom1.type
            tid2 = bond.atom2.type
            
            t = (tid1, tid2)
            bond.type = prm.bond_types[t]
    
            ## coordinates
            b1 = bond.atom1
            b1.xx, b1.xy, b1.xz = coords[b1.idx]
            
            b2 = bond.atom2
            b2.xx, b2.xy, b2.xz = coords[b2.idx]
            
            ## non bonded parameters eps and rmin
            name = b1.type
            eps, rmin, _, _ = prm.nonbonded_types[name]
            b1.rmin = rmin
            b1.epsilon = eps
            
            name = b2.type
            eps, rmin, _, _ = prm.nonbonded_types[name]
            b2.rmin = rmin
            b2.epsilon = eps
        
            
            
        ## initializing angle types
        for angle in angles:
            a1 = angle.atom1.type
            a2 = angle.atom2.type
            a3 = angle.atom3.type
            
            t = (a1, a2, a3)
            
            angle.type = prm.angle_types[t]
            a = angle.measure()
            a0 = angle.type
            
        for i, dihedral in enumerate(dihedrals):
            a1 = dihedral.atom1.type
            a2 = dihedral.atom2.type
            a3 = dihedral.atom3.type
            a4 = dihedral.atom4.type
            
            t = (a1, a2, a3, a4)
            g = self._get_bond_type(t, prm.dihedral_types)
            dihedrals[i].type = prm.dihedral_types[g]
            
        for i, improper in enumerate(impropers):
            a1 = improper.atom1.type
            a2 = improper.atom2.type
            a3 = improper.atom3.type
            a4 = improper.atom4.type
            
            t = (a1, a2, a3, a4)
            g = self._get_bond_type(t, prm.improper_types)
            impropers[i].type = prm.improper_types[g]

    @classmethod
    def bond_force(self, bond):
        b1 = bond.atom1
        b2 = bond.atom2
        coords = psf.coordinates

        r1 = coords[b1.idx]
        r2 = coords[b2.idx]
        r = r1 - r2
        r_val = np.linalg.norm(r)
        r_hat = r / r_val
        k = bond.type.k
        r0 = bond.type.req

        f_1 = -2 * k * (r_val - r0) * r_hat
        f_2 = -f_1
        return f_1, f_2
    
    @classmethod
    def angular_force(self, angle):
        coords = psf.coordinates
        theta = angle.measure()
        theta0 = angle.type.theteq
        k = angle.type.k

        a1 = angle.atom1
        a2 = angle.atom2
        a3 = angle.atom3

        a = coords[a1.idx]
        b = coords[a2.idx]
        c = coords[a3.idx]

        ba = (b - a)
        bc = (b - c)
        p_a = np.cross(ba, np.cross(ba, bc))
        p_a_hat = p_a / np.linalg.norm(p_a)
        f_a = - 2 * k * (theta - theta0) * p_a_hat / (np.linalg.norm(ba))

        p_c = np.cross(bc, np.cross(bc, ba))
        p_c_hat = p_c / np.linalg.norm(p_c)
        f_b = - 2 * k * (theta - theta0) * p_c_hat / (np.linalg.norm(bc))

        f_c = - f_a - f_b
        return f_a, f_b, f_c
        
    
    
    @classmethod
    def dihedral_force(self, dihed):
        coords = psf.coordinates
        dtype = dihed.type
        chi = dihed.measure() * (np.pi / 180)
        f_a, f_b, f_c, f_d = 0, 0, 0, 0
        
        if isinstance(dtype, parmed.DihedralType):
            n = dtype.per
            kchi = dtype.phi_k
            phase = dtype.phase * (np.pi / 180)

            a1 = dihed.atom1
            a2 = dihed.atom2
            a3 = dihed.atom3
            a4 = dihed.atom4

            a = coords[a1.idx]
            b = coords[a2.idx]
            c = coords[a3.idx]
            d = coords[a4.idx]

            ba = (b - a)
            bc = (b - c)
            p_1 = np.cross(ba, bc)
            p_1 /= np.linalg.norm(p_1)

            theta_1 = angle(a1, a2, a3) * (np.pi / 180)
            f_a = - k * np.sin(n * chi - phase) * n * p_1 / (np.linalg.norm(ab) * np.sin(theta_1))

            cd = (c - d)
            cb = -bc
            p_2 = np.cross(cd, cb)
            p_2 /= np.linalg.norm(p_2)

            theta_2 = angle(a2, a3, a4) * (np.pi / 180)
            f_d = - k * np.sin(n * chi - phase) * n * p_2 / (np.linalg.norm(cd) * np.sin(theta_2))

            o = (b + c) / 2
            oc = (o - c)
            t_c = - np.cross(oc, f_d) + 0.5 * np.cross(cd, f_d) + 0.5 * np.cross(ba, f_a)
            f_c = np.cross(t_c, oc) / (np.linalg.norm(cd)**2)


            f_b =  - f_a - f_c - f_d
            
        elif isinstance(dtype, parmed.DihedralTypeList):
            for term in dtype:
                n = term.per
                kchi = term.phi_k
                phase = term.phase * (np.pi / 180)

                a1 = dihed.atom1
                a2 = dihed.atom2
                a3 = dihed.atom3
                a4 = dihed.atom4

                a = coords[a1.idx]
                b = coords[a2.idx]
                c = coords[a3.idx]
                d = coords[a4.idx]

                ba = (b - a)
                bc = (b - c)
                p_1 = np.cross(ba, bc)
                p_1 /= np.linalg.norm(p_1)

                theta_1 = angle(a1, a2, a3) * (np.pi / 180)
                f_a = (-1)*(kchi * np.sin(n * chi - phase) * n * p_1) / (np.linalg.norm(ba) * np.sin(theta_1))


                cd = (c - d)
                cb = -bc
                p_2 = np.cross(cd, cb)
                p_2 /= np.linalg.norm(p_2)

                theta_2 = angle(a2, a3, a4) * (np.pi / 180)
                f_d = (-1)* kchi * np.sin(n * chi - phase) * n * p_2 / (np.linalg.norm(cd) * np.sin(theta_2))


                o = (b + c) / 2
                oc = (o - c)
                t_c = (-1)* np.cross(oc, f_d) + 0.5 * np.cross(cd, f_d) + 0.5 * np.cross(ba, f_a)
                f_c = np.cross(t_c, oc) / (np.linalg.norm(cd)**2)


                f_b =  - f_a - f_c - f_d
        else:
            print("Cant determine parameters for {}".format(dtype))
        return f_a, f_b, f_c, f_d
    
    @classmethod
    def improper_force(self, imp):
        coords = psf.coordinates
        psi = imp.measure() * (np.pi / 180)
        dtype = imp.type
        psi_0 = dtype.psi_eq * (np.pi / 180)
        kpsi = dtype.psi_k 

        a1 = imp.atom1
        a2 = imp.atom2
        a3 = imp.atom3
        a4 = imp.atom4
        
        a = coords[a1.idx]
        b = coords[a2.idx]
        c = coords[a3.idx]
        d = coords[a4.idx]

        ba = (b - a)
        bc = (b - c)
        p_1 = np.cross(ba, bc)
        p_1 /= np.linalg.norm(p_1)

        theta_1 = angle(a1, a2, a3) * (np.pi / 180)
        f_a = (-2)* kpsi * (psi - psi_0) * p_1 / (np.linalg.norm(ba) * np.sin(theta_1))


        cd = (c - d)
        cb = -bc
        p_2 = np.cross(cd, cb)
        p_2 /= np.linalg.norm(p_2)

        theta_2 = angle(a2, a3, a4) * (np.pi / 180)
        f_d = (-2)* kpsi * (psi - psi_0) * p_2 / (np.linalg.norm(cd) * np.sin(theta_2))


        o = (b + c) / 2
        oc = (o - c)
        t_c = (-1)* np.cross(oc, f_d) + 0.5 * np.cross(cd, f_d) + 0.5 * np.cross(ba, f_a)
        f_c = np.cross(t_c, oc) / (np.linalg.norm(cd)**2)


        f_b =  - f_a - f_c - f_d
        return f_a, f_b, f_c, f_d
    
                     
    ## Total forces calculation
    def bond_forces(self):
        bonds = psf.bonds
        coords = psf.coordinates
        force = np.zeros(coords.shape)
        for bond in bonds:
            f_1, f_2 = Forces.bond_force(bond)
            
            force[bond.atom1.idx] += f_1
            force[bond.atom2.idx] += f_2
        return force
    
    def angular_forces(self):
        coords = psf.coordinates
        force = np.zeros(coords.shape)
        angles = psf.angles
        
        for angle in angles:
            theta = angle.measure()
            theta0 = angle.type.theteq
            k = angle.type.k
            
            f_a, f_b, f_c = Forces.angular_force(angle)
            
            force[angle.atom1.idx] += f_a
            force[angle.atom2.idx] += f_b
            force[angle.atom3.idx] += f_c
        return force
    
    def dihedral_forces(self):
        dihedrals = psf.dihedrals
        force = np.zeros(psf.coordinates.shape)
        coords = psf.coordinates
        
        for dihed in dihedrals:
            dtype = dihed.type
            
            chi = dihed.measure() * (np.pi / 180)

            f_a, f_b, f_c, f_d = Forces.dihedral_force(dihed)
            force[dihed.atom1.idx] += f_a
            force[dihed.atom2.idx] += f_b
            force[dihed.atom3.idx] += f_c
            force[dihed.atom4.idx] += f_d
            
        return force
    
    def improper_forces(self):
        impropers = psf.impropers
        coords = psf.coordinates
        force = np.zeros(coords.shape)
        for imp in impropers:
            f_a, f_b, f_c, f_d = Forces.improper_force(imp)
            force[imp.atom1.idx] += f_a
            force[imp.atom2.idx] += f_b
            force[imp.atom3.idx] += f_c
            force[imp.atom4.idx] += f_d
            
        return force
    
    def LJ_forces(self):
        coords = psf.coordinates
        force = np.zeros(coords.shape)
        for i, a in enumerate(psf.atoms):
            for j, b in enumerate(psf.atoms):
                if j <= i: 
                    continue
                eps_ij = np.sqrt(a.epsilon * b.epsilon)
                r_ij = (a.rmin + b.rmin) / 2
    
                r1 = coords[a.idx]
                r2 = coords[b.idx]
                delta_r = r1 - r2
                r = np.linalg.norm(delta_r)
                ratio = r_ij / r
                
                f_1 = -24 * eps_ij * (2 * (ratio ** 12) - (ratio**6)) * delta_r / r
                f_2 = -f_1
                
                force[a.idx] += f_1
                force[b.idx] += f_2
        return force
        
    
#     def Elec(self):
#         energy = 0
#         coords = psf.coordinates
#         for i, a in enumerate(psf.atoms):
#             for j, b in enumerate(psf.atoms):
#                 if j <= i: 
#                     continue
#                 k = 8.9 * 1e9
#                 r1 = coords[a.idx]
#                 r2 = coords[b.idx]
#                 r = np.linalg.norm(r1 - r2)
#                 q1 = a.charge
#                 q2 = b.charge
#                 en = k * q1 * q2 / (r * 1e-9)
#         return energy
            
    ## DIHEDRAL AND IMPROPER HELPER FUNCTION
    def _get_bond_type(self, t, search_list):
        '''
        Input :
        t : a 4-atom tuple, 
        search_list: usually prm.dihedral_types or prm.improper_types

        Return type: BondType
        '''
        for dtype in search_list:
            val = True
            for i in range(4):
                val = val and (dtype[i] == t[i] or dtype[i] == 'X')

            if val:
                return dtype

            ## Attempt reverse search
            val = True
            for i in range(4):
                val = val and (dtype[i] == t[3 - i] or dtype[i] == 'X')

            if val:
                return dtype
        return None
    
    def bonded(self):
        return self.bond_forces() + self.angular_forces() + self.dihedral_forces() + self.improper_forces()

    def nonbonded(self):
        return self.LJ_forces()
    
    def all_forces(self):
        return self.bonded() + self.nonbonded()
    
    def update(self):
        coords = psf.coordinates
        
        for atom in psf.atoms:
            atom.xx, atom.xy, atom.xz = coords[atom.idx]

In [7]:
# forces = Forces(psf, pdb, prm)
# f_bond = forces.bond_forces()
# f_angle = forces.angular_forces()
# f_dihedral = forces.dihedral_forces()
# f_impropers = forces.improper_forces()
# f_LJ = forces.LJ_forces()

# Coding the velocity verlet integrator

In [ ]:
x = []
dt = 1e-3

v = np.random.rand(*psf.coordinates.shape)

m = 0
for atom in psf.atoms:
    m += atom.mass




x_orig = psf.coordinates.copy()

psf.coordinates = x_orig

x = psf.coordinates

F_k = forces.all_forces()
print(F_k.sum())
x_new = x + dt * v + (F_k * (dt**2) / (2 * m))
psf.coordinates = x_new
F_k1 = forces.all_forces()
print(np.linalg.norm(F_k1 - F_k))
print(psf.coordinates[0])
v = v + (dt * (F_k + F_k1) / (2 * m))

print(np.linalg.norm(x_new - x))

In [8]:

forces.update()

## Plotting forces for validation - Incomplete

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def plot_bond(bond):
    a1 = bond.atom1
    a2 = bond.atom2
    
    r1 = np.array([a1.xx, a1.xy, a1.xz])
    r2 = np.array([a2.xx, a2.xy, a2.xz])
    
    f_1, f_2 = Forces.bond_force(bond)

    f_1 /= (np.linalg.norm(f_1))
    f_2 /= (np.linalg.norm(f_2))

    fig = plt.figure(figsize = (8, 8))
    ax = fig.add_subplot(111,projection='3d')
    ax.scatter(a1.xx, a1.xy, a1.xz,  color='green', s = 100)
    ax.scatter(a2.xx, a2.xy, a2.xz,  color='blue', s = 100)
    
    ax.quiver(*r1, f_1[0], f_1[1], f_1[2], color = ['r'], arrow_length_ratio = 0.1)
    ax.quiver(*r2, f_2[0], f_2[1], f_2[2], color = ['g'], arrow_length_ratio = 0.1)
    ax.view_init(70, -10)
    r1 = np.array([a1.xx, a1.xy, a1.xz])
    r2 = np.array([a2.xx, a2.xy, a2.xz])

In [ ]:
plot_bond(psf.bonds[0])

In [ ]:
def plot_angle(angle):
    theta = angle.measure() * (np.pi / 180)
    theta0 = angle.type.theteq * (np.pi / 180)
    k = angle.type.k
    
    coords = psf.coordinates
    

    a1 = angle.atom1
    a2 = angle.atom2
    a3 = angle.atom3

    a = coords[a1.idx]
    b = coords[a2.idx]
    c = coords[a3.idx]

    ba = (b - a)
    bc = (b - c)

    p_c = np.cross(bc, np.cross(bc, ba))

    f_a, f_b, f_c = Forces.angular_force(angle)
    
    fig = plt.figure(figsize = (8, 8))
    ax = fig.add_subplot(111,projection='3d')
    ax.scatter(*a,  color='green', s = 100)
    ax.scatter(*b,  color='blue', s = 100)
    ax.scatter(*c,  color='red', s = 100)

    
    ax.quiver(*b, *(-1 * ba), color = ['g'], arrow_length_ratio = 0.05)
    ax.quiver(*b, *(-1 * bc), color = ['r'], arrow_length_ratio = 0.05)
    
    ax.quiver(*a, *f_a, color = ['y'], length = 0.01, arrow_length_ratio = 0.05)
    ax.quiver(*b, *f_b, color = ['y'], length = 0.005, arrow_length_ratio = 0.05)
    ax.quiver(*c, *f_c, color = ['y'], length = 0.005, arrow_length_ratio = 0.05)

In [ ]:
plot_angle(psf.angles[0])